# 39AA Project Part 2
[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/glaframb71/CS39AA-Project/blob/main/39aa-project.ipynb)


<font size=3>Welcome to Part 2 of my 39AA Project! In this notebook, we will be taking the dataset that was explored in Part 1 and trying to complete our task of predicting an athlete's football position based on their height, weight, and college. Let's get started by importing a few packages we will use in this notebook.</font>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE

<font size='3'>Now lets import the players data from the project GitHub.</font>

In [2]:
playersDataUrl = "https://raw.githubusercontent.com/glaframb71/CS39AA-Project/main/data/players.csv"
playersData = pd.read_csv(playersDataUrl)

playersData.head()

,nflId,height,weight,birthDate,collegeName,position,displayName
0,25511,6-4,225,1977-08-03,Michigan,QB,Tom Brady
1,29550,6-4,328,1982-01-22,Arkansas,T,Jason Peters
2,29851,6-2,225,1983-12-02,California,QB,Aaron Rodgers
3,30842,6-6,267,1984-05-19,UCLA,TE,Marcedes Lewis
4,33084,6-4,217,1985-05-17,Boston College,QB,Matt Ryan


<font size='3'>Great! The only thing missing from the DataFrame is a column that converts the 'height' column, which currently represents player heights in feet as strings, into a 'height_inches' column that represents player heights in inches as ints.</font>

In [3]:
# First we will add in the height_inches column to playersData and slot it next to the height column.
playersData = playersData.copy()
playersData.loc[:, 'height_inches'] = playersData['height'].apply(lambda x: int(x.split('-')[0]) * 12 + int(x.split('-')[1]))
height_index = playersData.columns.get_loc('height')
playersData.insert(height_index + 1, "height_inches", playersData.pop("height_inches"))

playersData.head()

,nflId,height,height_inches,weight,birthDate,collegeName,position,displayName
0,25511,6-4,76,225,1977-08-03,Michigan,QB,Tom Brady
1,29550,6-4,76,328,1982-01-22,Arkansas,T,Jason Peters
2,29851,6-2,74,225,1983-12-02,California,QB,Aaron Rodgers
3,30842,6-6,78,267,1984-05-19,UCLA,TE,Marcedes Lewis
4,33084,6-4,76,217,1985-05-17,Boston College,QB,Matt Ryan


<font size='3'>Now that the 'height_inches' column has been made, lets clean up the DataFrame to hold only the columns that are relevant to the task we are looking to accomplish. As a refresher, these columns will be 'height_inches', 'weight', 'collegeName', and 'position'.</font>

In [4]:
players = playersData[['height_inches', 'weight', 'collegeName', 'position']]

players.head()

,height_inches,weight,collegeName,position
0,76,225,Michigan,QB
1,76,328,Arkansas,T
2,74,225,California,QB
3,78,267,UCLA,TE
4,76,217,Boston College,QB


<font size='3'>Now that we have the players data that was explored in Part 1, we can recall that there was 2 positons that showed up only once in the data. These poisitions were 'DB' and 'LS'. I want to drop these two positions since there isn't very much data to represent these positions. We can do that with the following code.</font>

In [5]:
print('Players shape before filter:')
print(players.shape)
positions_to_drop = ['DB', 'LS']
filtered_players = players[~players['position'].isin(positions_to_drop)]
print('Players shape after filter:')
print(filtered_players.shape)

Players shape before filter:
(1683, 4)
Players shape after filter:
(1681, 4)


<font size='3'>We now have our DataFrame holding the relevant information that is needed. Let's split up our data into X and y training and validation sets using the method **train_test_split** imported from sklearn.model_selection.</font>

In [6]:
X = filtered_players.drop('position', axis=1)
y = filtered_players['position']
X.head()

,height_inches,weight,collegeName
0,76,225,Michigan
1,76,328,Arkansas
2,74,225,California
3,78,267,UCLA
4,76,217,Boston College


In [7]:
X = pd.get_dummies(X, columns=['collegeName'])

In [8]:
smote = SMOTE(sampling_strategy='not majority', k_neighbors=1)
X_sm, y_sm = smote.fit_resample(X,y)
print(X_sm.shape)
print(y_sm.shape)

(3706, 227)
(3706,)


In [9]:
X_train_raw, X_val_raw, y_train, y_val = train_test_split(X_sm, y_sm, test_size=0.20, random_state=42)

<font size='3'>Sweet! We have split our dataset into training and validation sets. Now, to make our data more compatible with machine learning algorithms, let's convert the 'collegeName' column in our X taining and validation sets into a numerical format using one-hot encoding.</font>

In [10]:
# Import necessary libraries
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Initialize the Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=5)

# Fit the model using the training data
gb_clf.fit(X_train_raw, y_train)

# Use the trained model to make predictions on the training data
y_train_pred = gb_clf.predict(X_train_raw)

# Calculate the training accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)

# Print the training accuracy
print("Training Accuracy: ", train_accuracy)

# Use the trained model to make predictions on the validation data
y_val_pred = gb_clf.predict(X_val_raw)

# Calculate the validation accuracy
val_accuracy = accuracy_score(y_val, y_val_pred)

# Print the validation accuracy
print("Validation Accuracy: ", val_accuracy)

Training Accuracy:  0.7419028340080972
Validation Accuracy:  0.5242587601078167


In [11]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 4, 5],
    'subsample': [0.5, 0.7, 1.0],
    'max_features': ['sqrt', 'log2', None]
}

# Initialize the Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier(random_state=5)

# Initialize the GridSearchCV object
grid_search = GridSearchCV(estimator=gb_clf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Fit the GridSearchCV object to the data
grid_search.fit(X_train_raw, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters: ", best_params)

# Get the best estimator
best_gb_clf = grid_search.best_estimator_

# Use the best estimator to make predictions on the training data
y_train_pred = best_gb_clf.predict(X_train_raw)

# Calculate the training accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)

# Print the training accuracy
print("Training Accuracy: ", train_accuracy)

# Use the best estimator to make predictions on the validation data
y_val_pred = best_gb_clf.predict(X_val_raw)

# Calculate the validation accuracy
val_accuracy = accuracy_score(y_val, y_val_pred)

# Print the validation accuracy
print("Validation Accuracy: ", val_accuracy)


Fitting 5 folds for each of 162 candidates, totalling 810 fits
[CV] END learning_rate=0.01, max_depth=3, max_features=sqrt, n_estimators=50, subsample=0.5; total time=   2.4s
[CV] END learning_rate=0.01, max_depth=3, max_features=sqrt, n_estimators=50, subsample=0.7; total time=   2.9s
[CV] END learning_rate=0.01, max_depth=3, max_features=sqrt, n_estimators=50, subsample=1.0; total time=   2.6s
[CV] END learning_rate=0.01, max_depth=3, max_features=sqrt, n_estimators=100, subsample=0.5; total time=   4.8s
[CV] END learning_rate=0.01, max_depth=3, max_features=sqrt, n_estimators=100, subsample=0.5; total time=   4.7s
[CV] END learning_rate=0.01, max_depth=3, max_features=sqrt, n_estimators=100, subsample=0.7; total time=   4.9s
[CV] END learning_rate=0.01, max_depth=3, max_features=sqrt, n_estimators=100, subsample=1.0; total time=   5.0s
[CV] END learning_rate=0.01, max_depth=3, max_features=sqrt, n_estimators=200, subsample=0.5; total time=   9.6s
[CV] END learning_rate=0.01, max_dep